In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY_NEW")

## Defining tool

In [3]:
from groq import Groq
import json

client = Groq()
MODEL = "llama3-8b-8192"

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [4]:
def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculator function to perform calcuation."
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate", # function name need to match to our actual function name
                "description": "Evaluate a mathematical expression", # description of our function
                "parameters": { # parameter to our function
                    "type": "object",
                    "properties": {
                        "expression": { # parameter name
                            "type": "string",
                            "description": "The mathematical expression to evaluate"
                        }
                    },
                    "required": ["expression"]
                }
            }
        }
    ]
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=False,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )
    
    response_message = response.choices[0].message
    print(f"Initial Response Message: {response_message}\n")
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        print("Tool call is available...\n")
        available_functions = {
            "calculate": calculate
        }
        
        messages.append(response_message)
        
        for tool_call in tool_calls:
            print(f"Tool call: {tool_call}\n")
            function_name = tool_call.function.name
            print(f"Using function {function_name} for {user_prompt}")
            function_to_call = available_functions.get(function_name)
            function_arguments = json.loads(tool_call.function.arguments)
            print(f"Function argument: {function_arguments}\n")
            
            function_response = function_to_call(
                expression=function_arguments.get("expression")
            )
            print(f"Got the response from the function {function_response}")
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
            
        print(f"Final message to the llm after function call : {messages}")
            
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
        )
        
        return second_response.choices[0].message.content
    else:
        print("No tool call needed...\n")
        return response_message.content

In [5]:
run_conversation("what is 2 + 2?")

Initial Response Message: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_36v2', function=Function(arguments='{"expression":"2 + 2"}', name='calculate'), type='function')])

Tool call is available...

Tool call: ChatCompletionMessageToolCall(id='call_36v2', function=Function(arguments='{"expression":"2 + 2"}', name='calculate'), type='function')

Using function calculate for what is 2 + 2?
Function argument: {'expression': '2 + 2'}

Got the response from the function {"result": 4}
Final message to the llm after function call : [{'role': 'system', 'content': 'You are a calculator assistant. Use the calculator function to perform calcuation.'}, {'role': 'user', 'content': 'what is 2 + 2?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_36v2', function=Function(arguments='{"expression":"2 + 2"}', name='calculate'), type='function')]), 

'The result of the calculation is indeed 4.'

In [6]:
response = run_conversation("What is banana?")

print(response)

Initial Response Message: ChatCompletionMessage(content="A banana is a type of long, curved fruit that grows in clusters on banana plants. It is typically yellow when it's ripe and has a sweet, creamy flavor. Bananas are a good source of several important nutrients, including potassium, vitamins C and B6, and fiber.", role='assistant', function_call=None, tool_calls=None)

No tool call needed...

A banana is a type of long, curved fruit that grows in clusters on banana plants. It is typically yellow when it's ripe and has a sweet, creamy flavor. Bananas are a good source of several important nutrients, including potassium, vitamins C and B6, and fiber.


## Routing

In [7]:
ROUTING_MODEL = "llama3-70b-8192"
TOOL_USE_MODEL = "llama3-groq-70b-8192-tool-use-preview"
GENERAL_MODEL = "llama3-70b-8192"

In [8]:
def calculate(expression):
    """Tool to evaluate a mathematical expression"""
    print("model is using calculate tool")
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

In [9]:
def route_query(query):
    """Routing logic to let LLM decide if tools are needed"""
    routing_prompt = f"""
    Given the following user query, determine if any tools are needed to answer it.
    If a calculator tool is need, response with 'TOOL: CALCULATE'.
    If no tools are needed, responsd with 'NO TOOL'
    
    User Query: {query}
    
    Response:\n"""
    
    response = client.chat.completions.create(
        model=ROUTING_MODEL,
        messages=[
            {"role": "system", "content": "you are a routing assistant. Determine if tools are needed to answer the user query."},
            {"role": "user", "content": routing_prompt}
        ],
        max_tokens=20
    )
    
    routing_decision = response.choices[0].message.content.strip()
    
    if "TOOL: CALCULATE" in routing_decision:
        return "calculate tool needed"
    else:
        return "no tool needed to answer the query"

In [10]:
def run_with_tool(query):
    """Use the tool to perform the calculation"""
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculator function to perform calcuation."
        },
        {
            "role": "user",
            "content": query
        }
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate"
                        }
                    },
                    "required": ["expression"]
                }
            }
        }
    ]
    
    response = client.chat.completions.create(
        model=TOOL_USE_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=3000
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        print("Tool call use available...\n")
        messages.append(response_message)
        
        for tool_call in tool_calls:
            print(f"tool call: {tool_call}\n")
            function_args = json.loads(tool_call.function.arguments)
            function_response = calculate(function_args.get("expression"))
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": "calculate",
                "content": function_response
            })
            
        second_response = client.chat.completions.create(
            model=TOOL_USE_MODEL,
            messages=messages,
        )
        return second_response.choices[0].message.content
    else:
        print("No need to use tool...\n")
        return response_message.content

In [11]:
def run_general(query):
    """Use the general model to answer the query since no tool needed to answer to answer the query"""
    
    response = client.chat.completions.create(
        model=GENERAL_MODEL,
        messages=[
            {
                "role": "system",
                "content": "Yo are a helpful assistant answer the user query."
            },
            {
                "role": "user",
                "content": query
            }
        ]
    )
    
    return response.choices[0].message.content

In [12]:
# run the main function to determine if tool to use or not and if needed use to answer the user query.
def process_query(query):
    """process the query and route it to the appropriate model"""
    route = route_query(query)
    if "calculate" in route:
        response = run_with_tool(query)
    else:
        response = run_general(query)
    return {
        "query": query,
        "route": route,
        "response": response
    }

In [13]:
queries = [
    "what is the capital of France?",
    "calculate 25 * 4 + 10 + 90"
]

for query in queries:
    print("...\n")
    print(process_query(query))

...

{'query': 'what is the capital of France?', 'route': 'no tool needed to answer the query', 'response': 'The capital of France is Paris!'}
...

Tool call use available...

tool call: ChatCompletionMessageToolCall(id='call_vwy1', function=Function(arguments='{"expression": "25 * 4 + 10 + 90"}', name='calculate'), type='function')

model is using calculate tool
{'query': 'calculate 25 * 4 + 10 + 90', 'route': 'calculate tool needed', 'response': 'The result of the calculation is 200.'}


## Parallel tool use

In [14]:
def get_temperature(location: str):
    temperatures = {
        "Pokhara": 10,
        "Kathmandu": 11
    }
    return temperatures.get(location, "Temperature data not available")

def get_weather_condition(location: str):
    conditions = {
        "Pokhara": "Cloudy",
        "Kathmandu": "Sunny"
    }
    return conditions.get(location, "Weather condition data not available")

In [15]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful weather assistant."
    },
    {
        "role": "user",
        "content": "What is the weather like in Kathmandu and Pokhara?"
    }
]

tools = [
    # tool1
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description":"Get the temperature of a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city"
                    }
                },
                "required": ["location"],
            },
        },
    },
    # tool2
    {
        "type": "function",
        "function": {
            "name": "get_weather_condition",
            "description": "Get the weather condition of a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the location."
                    }
                },
                "required": ["location"]
            }
        }
    }
]

In [16]:
# making the call
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=5000
)

print(f"Initial response: {response}")

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

messages.append(response_message)

available_functions_for_weather = {
    "get_temperature": get_temperature,
    "get_weather_condition": get_weather_condition
}

for tool_call in tool_calls:
    print(f"Tool: {tool_call}")
    function_name = tool_call.function.name
    function_to_call = available_functions_for_weather.get(function_name)
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        **function_args
    )
    
    messages.append(
        {
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        }
    )
    
response_after_tool_call = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=3000
)

print(response_after_tool_call.choices[0].message.content)

Initial response: ChatCompletion(id='chatcmpl-3123a231-edc7-4cee-a929-9cc36db4440e', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_53sy', function=Function(arguments='{"location":"Kathmandu"}', name='get_weather_condition'), type='function'), ChatCompletionMessageToolCall(id='call_a5aw', function=Function(arguments='{"location":"Pokhara"}', name='get_weather_condition'), type='function')]))], created=1732812185, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_179b0f92c9', usage=CompletionUsage(completion_tokens=120, prompt_tokens=1042, total_tokens=1162, completion_time=0.1, prompt_time=0.05890477, queue_time=0.003367289999999995, total_time=0.15890477), x_groq={'id': 'req_01jdstf81he38r17k3cnt1bfq4'})
Tool: ChatCompletionMessageToolCall(id='call_53sy', function=Function(arguments='{"location":"Kathmandu"}', 

In [ ]:
import instructor
from pydantic import BaseModel, Field
from groq import Groq

# Define the tool schema
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)"
            }
        },
        "required": ["location"]
    }
}

# Define the Pydantic model for the tool call
class ToolCall(BaseModel):
    input_text: str = Field(description="The user's input text")
    tool_name: str = Field(description="The name of the tool to call")
    tool_parameters: str = Field(description="JSON string of tool parameters")

class ResponseModel(BaseModel):
    tool_calls: list[ToolCall]

# Patch Groq() with instructor
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

def run_conversation(user_prompt):
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"You are an assistant that can use tools. You have access to the following tool: {tool_schema}"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        response_model=ResponseModel,
        messages=messages,
        temperature=0.7,
        max_tokens=1000,
    )

    return response.tool_calls

# Example usage
user_prompt = "What's the weather like in San Francisco?"
tool_calls = run_conversation(user_prompt)

for call in tool_calls:
    print(f"Input: {call.input_text}")
    print(f"Tool: {call.tool_name}")
    print(f"Parameters: {call.tool_parameters}")
    print()

AttributeError: module 'pydantic._internal._generate_schema' has no attribute 'VALIDATE_CALL_SUPPORTED_TYPES'